stopword removal helps

Ranking using just answers is not useful. Ranking should be done on basis of both answer and question - ranking with just question/answer does not give good results

### Final Runs

The plan is to submit 5 runs. 

The best model so far has been -

- Get similar questions using tfidf similarity
- Use the accepted and answers from those questions as candidates
- Rank them using arqmath-roberta-base-2M

5 runs :
- Secondary : BERT-large
- Primary - Sim questions : RoBERTa-fintuned-2M or 3M
- tf-idf Baseline
- 

Analysis of candidates from similar questions vs answer retreival 

## Rank questions on the basis of BERT

We have seen that similarity on the basis of answers doesnt work as -
- the way the ground is created
- BERT finetuning increases rank


Some oberservation - 

- Similar questions increase recall and are able to get a lot more similar 
- np.mean gives better results than the CLS token representation

New Experiments - 

Check results for search against answer index and question index - how much overlap do each of them have ?


#### We should know the length of accepted answers so that we understand what is acceptable

Removing stop words helps but not that much

## An answer is not similar to the question ! Hence it should be ranked in a way that is different : inter-answer ranking - average the top-5 answers and then rank top-1000 based on those top-5

#### Reranking just the top 500 gives better BPREF but not better NDCG

link for training - https://tensorboard.dev/experiment/Ao82opUOTnamazhnHrjzSQ/

In [3]:
import jsonlines
import re
from tqdm.notebook import tqdm

In [4]:
runs_path = '/data/szr207/projects/ArqMath/runs/'

In [5]:
qrel = '/data/szr207/dataset/ArqMath/Task1/Sample Topics/qrel.V1.0.tsv'

In [13]:
from trectools import TrecQrel, TrecRun, TrecEval
import os

# A typical evaluation workflow
r1 = TrecRun(os.path.join(runs_path, "es.500.sim.tfidf.fintune-2M.dat"))
r1.topics()[:5] # Shows the first 5 topics: 601, 602, 603, 604, 605

qrels = TrecQrel(qrel)

te = TrecEval(r1, qrels)
rbp, residuals = te.get_rbp()           # RBP: 0.474, Residuals: 0.001
p100 = te.get_precision(depth=100)     # P@100: 0.186

# Check if documents retrieved by the system were judged:
cover10 = r1.get_mean_coverage(qrels, topX=10)   # 9.99
cover1000 = r1.get_mean_coverage(qrels, topX=1000) # 481.390 
# On average for system 'input.aplrob03a' participating in robust03, 480 documents out of 1000 were judged.
print("Average number of documents judged among top 10: %.2f, among top 1000: %.2f" % (cover10, cover1000))


Average number of documents judged among top 10: 0.67, among top 1000: 18.00


In [7]:
from trectools import TrecRun, TrecEval, fusion

r1 = TrecRun(os.path.join(runs_path, "es.500.sim.tfidf.fintune-3M.dat"))
r2 = TrecRun(os.path.join(runs_path, "es.500.sim.ques.morelikethis.dat"))

# Easy way to create new baselines by fusing existing runs:
fused_run = fusion.reciprocal_rank_fusion([r1,r2])
r1_p25 = TrecEval(r1, qrels).get_ndcg()          # P@25: 0.3392
r2_p25 = TrecEval(r2, qrels).get_ndcg()          # P@25: 0.2872
fused_run_p25 = TrecEval(fused_run, qrels).get_ndcg()   # P@25: 0.3436

print("P@25 -- Run 1: %.3f, Run 2: %.3f, Fusion Run: %.3f" % (r1_p25, r2_p25, fused_run_p25))

# Save run to disk with all its topics
fused_run.print_subset("rrf.bm25.f3M.mlt.txt", topics=fused_run.topics())

NameError: name 'qrels' is not defined

In [16]:
from trectools import TrecRun, TrecEval, fusion

r1 = TrecRun(os.path.join(runs_path, "es.500.sim.tfidf.fintune-2M.dat"))
r2 = TrecRun(os.path.join(runs_path, "es.500.sim.ques.morelikethis.dat"))

# Easy way to create new baselines by fusing existing runs:
# fused_run = fusion.rank_biased_precision_fusion([r1,r2])
r1_p25 = TrecEval(r1, qrels).get_precision(depth=25)          # P@25: 0.3392
r2_p25 = TrecEval(r2, qrels).get_precision(depth=25)          # P@25: 0.2872
# fused_run_p25 = TrecEval(fused_run, qrels).get_precision(depth=25)   # P@25: 0.3436

# print("P@25 -- Run 1: %.3f, Run 2: %.3f, Fusion Run: %.3f" % (r1_p25, r2_p25, fused_run_p25))

# Save run to disk with all its topics
fusion.rank_biased_precision_fusion([r1,r2]).print_subset("rank_biased_precision_fusion.txt", topics=fused_run.topics())

A.101 Q0 1167363 1 0.202882 rank_biased_precision_fusion_p=0.800
A.101 Q0 2094402 2 0.200000 rank_biased_precision_fusion_p=0.800
A.101 Q0 630787 3 0.160000 rank_biased_precision_fusion_p=0.800
A.101 Q0 816003 4 0.160000 rank_biased_precision_fusion_p=0.800
A.101 Q0 290081 5 0.128000 rank_biased_precision_fusion_p=0.800
A.101 Q0 630790 6 0.128000 rank_biased_precision_fusion_p=0.800
A.101 Q0 1724427 7 0.102421 rank_biased_precision_fusion_p=0.800
A.101 Q0 1526692 8 0.102400 rank_biased_precision_fusion_p=0.800
A.101 Q0 46739 9 0.081920 rank_biased_precision_fusion_p=0.800
A.101 Q0 2754579 10 0.081920 rank_biased_precision_fusion_p=0.800
A.101 Q0 1912854 11 0.065536 rank_biased_precision_fusion_p=0.800
A.101 Q0 2947503 12 0.065536 rank_biased_precision_fusion_p=0.800
A.101 Q0 106518 13 0.052429 rank_biased_precision_fusion_p=0.800
A.101 Q0 3038867 14 0.052429 rank_biased_precision_fusion_p=0.800
A.101 Q0 1267661 15 0.041943 rank_biased_precision_fusion_p=0.800
A.101 Q0 1814550 16 0.0419

AttributeError: 'NoneType' object has no attribute 'print_subset'

In [20]:
from trectools import TrecRun, TrecEval, fusion

r1 = TrecRun(os.path.join(runs_path, "es.final.dat"))
r2 = TrecRun(os.path.join(runs_path, "es.500.sim.ques.morelikethis.dat"))

# Easy way to create new baselines by fusing existing runs:
fused_run = fusion.reciprocal_rank_fusion([r1,r2])
r1_p25 = TrecEval(r1, qrels).get_ndcg()          # P@25: 0.3392
r2_p25 = TrecEval(r2, qrels).get_ndcg()          # P@25: 0.2872
fused_run_p25 = TrecEval(fused_run, qrels).get_ndcg()   # P@25: 0.3436

print("NDCG -- Run 1: %.3f, Run 2: %.3f, Fusion Run: %.3f" % (r1_p25, r2_p25, fused_run_p25))

# Save run to disk with all its topics
fused_run.print_subset("rrf.bm25.mlt.txt", topics=fused_run.topics())

NDCG -- Run 1: 0.098, Run 2: 0.222, Fusion Run: 0.185
File rrf.bm25.tfidf.txt writen.


In [31]:
from trectools import TrecRun, TrecEval, fusion

r1 = TrecRun(os.path.join(runs_path, "es.500.sim.tfidf.fintune-3M.dat"))
# r2 = TrecRun(os.path.join(runs_path, "es.500.sim.tfidf.fintune-2M.dat"))
r3 = TrecRun(os.path.join(runs_path, "es.500.sim.ques.morelikethis.dat"))

# Easy way to create new baselines by fusing existing runs:
fused_run = fusion.reciprocal_rank_fusion([r1,r3])
r1_p25 = TrecEval(r1, qrels).get_ndcg()          # P@25: 0.3392
r2_p25 = TrecEval(r2, qrels).get_ndcg()          # P@25: 0.2872
fused_run_p25 = TrecEval(fused_run, qrels).get_ndcg()   # P@25: 0.3436

print("P@25 -- Run 1: %.3f, Run 2: %.3f, Fusion Run: %.3f" % (r1_p25, r2_p25, fused_run_p25))

# Save run to disk with all its topics
fused_run.print_subset("rrf.mlt.f3M.txt", topics=fused_run.topics())

P@25 -- Run 1: 0.229, Run 2: 0.228, Fusion Run: 0.238
File rrf.mlt.f3M.txt writen.


In [36]:
from trectools import TrecRun, TrecEval, fusion

r1 = TrecRun(os.path.join(runs_path, "es.rr.finetune.2M.dat"))
# r2 = TrecRun(os.path.join(runs_path, "es.500.sim.tfidf.fintune-2M.dat"))
r3 = TrecRun(os.path.join(runs_path, "rrf.tfidf.f3M.mlt.txt"))

# Easy way to create new baselines by fusing existing runs:
fused_run = fusion.reciprocal_rank_fusion([r1,r3])
r1_p25 = TrecEval(r1, qrels).get_ndcg()          # P@25: 0.3392
r2_p25 = TrecEval(r3, qrels).get_ndcg()          # P@25: 0.2872
fused_run_p25 = TrecEval(fused_run, qrels).get_ndcg()   # P@25: 0.3436

print("P@25 -- Run 1: %.3f, Run 2: %.3f, Fusion Run: %.3f" % (r1_p25, r2_p25, fused_run_p25))

# Save run to disk with all its topics
fused_run.print_subset("rrf.rrf.es-f2M.mlt-f3m.txt", topics=fused_run.topics())

P@25 -- Run 1: 0.097, Run 2: 0.238, Fusion Run: 0.210
File rrf.rrf.es-f2M.mlt-f3m.txt writen.


In [12]:
from trectools import TrecRun, TrecEval, fusion, TrecQrel
import os
r1 = TrecRun(os.path.join(runs_path, "es.mlt.qa-baseline.dat"))
# r2 = TrecRun(os.path.join(runs_path, "es.500.sim.tfidf.fintune-2M.dat"))
r3 = TrecRun(os.path.join(runs_path, "es.mlt.qai.rev.onlyques3M.dat"))
qrels = TrecQrel(qrel)

# Easy way to create new baselines by fusing existing runs:
fused_run = fusion.reciprocal_rank_fusion([r1,r3])
r1_p25 = TrecEval(r1, qrels).get_ndcg()          # P@25: 0.3392
r2_p25 = TrecEval(r3, qrels).get_ndcg()          # P@25: 0.2872
fused_run_p25 = TrecEval(fused_run, qrels).get_ndcg()   # P@25: 0.3436

print("P@25 -- Run 1: %.3f, Run 2: %.3f, Fusion Run: %.3f" % (r1_p25, r2_p25, fused_run_p25))

# Save run to disk with all its topics
fused_run.print_subset("rrf.qa-baseline.qai.onlyques.rr.3M.txt", topics=fused_run.topics())

P@25 -- Run 1: 0.263, Run 2: 0.256, Fusion Run: 0.261
File rrf.qa-baseline.qai.onlyques.rr.3M.txt writen.


In [10]:
from trectools import TrecRun, TrecEval, fusion, TrecQrel
import os
r1 = TrecRun(os.path.join(runs_path, "es.mlt.qa-baseline.dat"))
# r2 = TrecRun(os.path.join(runs_path, "es.500.sim.tfidf.fintune-2M.dat"))
r3 = TrecRun(os.path.join(runs_path, "es.mlt.qai.merged3M.dat"))
qrels = TrecQrel(qrel)

# Easy way to create new baselines by fusing existing runs:
fused_run = fusion.reciprocal_rank_fusion([r1,r3])
r1_p25 = TrecEval(r1, qrels).get_ndcg()          # P@25: 0.3392
r2_p25 = TrecEval(r3, qrels).get_ndcg()          # P@25: 0.2872
fused_run_p25 = TrecEval(fused_run, qrels).get_ndcg()   # P@25: 0.3436

print("P@25 -- Run 1: %.3f, Run 2: %.3f, Fusion Run: %.3f" % (r1_p25, r2_p25, fused_run_p25))

# Save run to disk with all its topics
fused_run.print_subset("rrf.qa-baseline.qai.3M.txt", topics=fused_run.topics())

P@25 -- Run 1: 0.263, Run 2: 0.255, Fusion Run: 0.267
File rrf.qa-baseline.qai.3M.txt writen.


#### Checking overlap and exclusive post_id of relevant posts

- choose best run from searching the answer index
- choose best run from searching the question index

In [44]:
ans_run = os.path.join(runs_path,'es.rr.1k.dat')
ques_run = os.path.join(runs_path,'es.500.sim.tfidf.fintune-3M.dat')

In [48]:
from collections import defaultdict

dict_ans = defaultdict(list)

with open(ans_run, 'r') as eval_file:
    for line in eval_file.readlines():
#     line = eval_file.read()
        list_line = line.split('\t')
        dict_ans[list_line[0]].append(list_line[2])

In [50]:
dict_ques = defaultdict(list)

with open(ques_run, 'r') as eval_file:
    for line in eval_file.readlines():
#     line = eval_file.read()
        list_line = line.split('\t')
        dict_ques[list_line[0]].append(list_line[2])

In [51]:
dict_rel = defaultdict(list)

with open(qrel, 'r') as eval_file:
    for line in eval_file.readlines():
#     line = eval_file.read()
        list_line = line.split('\t')
        dict_rel[list_line[0]].append(list_line[2])

In [57]:
def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 

def Diff(li1, li2): 
    return (list(set(li1) - set(li2))) 

In [66]:
for key in dict_rel:
    print("Answer Run:" , key, intersection(dict_ans[key], dict_rel[key]), len(intersection(dict_ans[key], dict_rel[key])))
    print("Question Run:" , key, intersection(dict_ques[key], dict_rel[key]), len(intersection(dict_ques[key], dict_rel[key])) )
    print("Diff Ans and Ques:", key, Diff(intersection(dict_ans[key], dict_rel[key]), intersection(dict_ques[key], dict_rel[key])), len(Diff(intersection(dict_ans[key], dict_rel[key]), intersection(dict_ques[key], dict_rel[key]))) )
    print("Diff Ans and Ques:", key, Diff(intersection(dict_ques[key], dict_rel[key]), intersection(dict_ans[key], dict_rel[key])), len(Diff(intersection(dict_ques[key], dict_rel[key]), intersection(dict_ans[key], dict_rel[key]))))
    print("===============================================================")

Answer Run: A.31 ['2170920', '48202', '48219', '1623400', '48164', '48165', '616514', '48181', '2227543', '53790', '692232', '1596444', '70739', '168286', '70741', '48167', '743738'] 17
Question Run: A.31 ['2170920', '472635', '574514', '48165', '2362771', '48164', '48219', '53784', '69435', '48202', '1609339', '2227543', '2602592', '2968174', '692232', '1596444', '672516', '168290', '743738', '1623400', '48172', '168286', '70739', '53779', '53790', '48235', '48181', '1639289', '48167', '70741', '48260', '897705', '860842', '48241', '1018719', '48162', '53781', '98328', '168305', '982759', '48184', '168323'] 42
Diff Ans and Ques: A.31 ['616514'] 1
Diff Ans and Ques: A.31 ['48260', '1639289', '168305', '2362771', '2602592', '2968174', '48184', '168323', '1609339', '860842', '98328', '982759', '48172', '574514', '897705', '48235', '168290', '53784', '48241', '472635', '1018719', '48162', '69435', '53781', '672516', '53779'] 26
Answer Run: A.78 ['2008712', '2008609'] 2
Question Run: A.78 

In [73]:
ques_path = '/data/szr207/dataset/ArqMath/jsons/questions/all.ques.jsonl'

dict_ques = {}

with jsonlines.open(os.path.join(ques_path)) as reader:
        for obj in tqdm(reader):
            dict_ques[obj['post_id']] = obj
            
ans_path = '/data/szr207/dataset/ArqMath/jsons/answers/all.ans.jsonl'
      
dict_ans = {}
    
with jsonlines.open(os.path.join(ans_path)) as reader:
        for obj in tqdm(reader):
            dict_ans[obj['post_id']] = obj

dict_aid_post = {}
with jsonlines.open(os.path.join(ans_path)) as reader:
    for obj in tqdm(reader):
        if obj['body']:
            obj['body'] = re.sub('<[^<]+?>', '',  obj['body'])
            dict_aid_post[obj['post_id']] = obj['body']